In [4]:
import pandas as pd
from pathlib import Path

In [8]:
src = Path('/home/t9s9/Datasets/TAUVideos/')
trg = Path('/home/t9s9/Datasets/TAUVideos/videos')

for dir in src.iterdir():
    if dir.name.startswith('TAU'):
        for file in (dir / 'video').iterdir():
            if file.name.endswith('.mp4'):
                file.rename(trg / file.name)
        (dir / 'video').rmdir()
        dir.rmdir()

In [ ]:
df = pd.read_csv('/media/t9s9/SSD_ubuntu/datasets/TAU/annot/meta.csv', sep='\t')
df['category'] = df.scene_label.astype('category')
df['target'] = df.category.cat.codes
df

In [16]:
root = Path('/media/t9s9/SSD_ubuntu/datasets/TAU/audio')
df['filename'] = df['filename_audio'].apply(lambda x: str(Path(x).with_suffix('.flac').name))
exists = df['filename'].apply(lambda x: (root / x).exists())
print(f'Number of files that exist: {exists.sum()} / {len(exists)}')
df = df[exists]
df = df.drop(columns=['filename_audio', 'filename_video', 'scene_label', 'identifier'])
df

Number of files that exist: 12291 / 12291


,category,target,filename
0,airport,0,airport-lisbon-1000-40000.flac
1,bus,1,bus-lyon-1001-40001.flac
2,shopping_mall,6,shopping_mall-lisbon-1002-40002.flac
3,street_pedestrian,7,street_pedestrian-lyon-1003-40003.flac
4,street_pedestrian,7,street_pedestrian-lisbon-1004-40004.flac
...,...,...,...
12286,tram,9,tram-vienna-285-8635.flac
12287,tram,9,tram-vienna-285-8636.flac
12288,tram,9,tram-vienna-285-8637.flac
12289,tram,9,tram-vienna-285-8638.flac


In [17]:
df.to_parquet('/media/t9s9/SSD_ubuntu/datasets/TAU/annot.parquet', index=False)

In [11]:
idx_to_label = df[['target', 'category']].drop_duplicates().sort_values('target').set_index('target').to_dict()[
    'classnames']
idx_to_label

{0: 'airport',
 1: 'bus',
 2: 'metro',
 3: 'metro_station',
 4: 'park',
 5: 'public_square',
 6: 'shopping_mall',
 7: 'street_pedestrian',
 8: 'street_traffic',
 9: 'tram'}

In [13]:
list(map(lambda x: x.replace('_', ' ').lower(), idx_to_label.values()))

['airport',
 'bus',
 'metro',
 'metro station',
 'park',
 'public square',
 'shopping mall',
 'street pedestrian',
 'street traffic',
 'tram']

In [2]:
from MVALM.datasets import TAU

In [3]:
dataset = TAU(datasets_root='/media/t9s9/SSD_ubuntu/datasets')
dataset

TAU(split=test, len=12291, label=classification)

In [3]:
string = "https://zenodo.org/record/4477542/files/TAU-urban-audio-visual-scenes-2021-development.video.{}.zip?download=1"
with open('urls.txt', 'w') as f:
    for i in range(1, 17):
        f.write(string.format(i) + '\n')